In [85]:
from sympy import *
from sympy.physics.hep.gamma_matrices import GammaMatrix as G
from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum.state import Ket, Bra
from sympy.physics.quantum.operator import Operator
from sympy.physics.quantum.operator import DifferentialOperator
#from sympy.physics.paulialgebra import Pauli, evaluate_pauli_product

In [124]:
s0 = Matrix([[1, 0],[0, 1]]); s1 = Matrix([[0, 1],[1, 0]]); s2 = Matrix([[0, I],[-I, 0]]); s3 = Matrix([[1, 0],[0, -1]]);
kx, ky, kz, vf, J, hbar = symbols("kx ky kz vf J hbar",real=True)
x, y, z = symbols("x y z", real=True)

In [108]:
del D

In [125]:
HB = J*(cos(x)*s2+sin(x)*s3)
display(HB)
display(Hweyl)

Matrix([
[   J*sin(x), I*J*cos(x)],
[-I*J*cos(x),  -J*sin(x)]])

Matrix([
[           -I*hbar*vf*D(z), -I*hbar*vf*(D(x) + I*D(y))],
[-I*hbar*vf*(D(x) - I*D(y)),             I*hbar*vf*D(z)]])

In [127]:
P, eigs = HB.diagonalize(normalize=True)
x, y, z = symbols("x y z", commutative=False)
P = simplify(P); display(P)

Matrix([
[-I*cos(x)/((sin(x) + 1)*sqrt(Abs(cos(x)/(sin(x) + 1))**2 + 1)), -I*cos(x)/((sin(x) - 1)*sqrt(Abs(cos(x)/(sin(x) - 1))**2 + 1))],
[                       1/sqrt(Abs(cos(x)/(sin(x) + 1))**2 + 1),                        1/sqrt(Abs(cos(x)/(sin(x) - 1))**2 + 1)]])

In [122]:
ddx = D(x); ddy = D(y); ddz = D(z); 
Hweyl = -I*hbar*vf*(ddx*s1+ddy*s2+ddz*s3)

In [123]:
simplify(P**-1@Hweyl@P)

Matrix([
[                                                                    I*hbar*vf*(sin(x)**2*D(z) + sin(x)*cos(x)*D(y) + sin(x)*D(z) + cos(x)*D(y))/(sin(x) + 1), hbar*vf*(-I*sin(x)**2*D(y)/cos(x) + I*sin(x)*D(z) + tan(x)*D(x) - I*tan(x)*D(y) + I*D(z) + D(x)/cos(x))],
[hbar*vf*cos(x)*(-(sin(x) + 1)**2*(D(x) + I*D(y)) + I*(sin(x) + 1)*cos(x)*D(z) + I*cos(x)*((sin(x) + 1)*D(z) + I*cos(x)*(D(x) - I*D(y))))/(2*(sin(x) + 1)**2),                                            -I*hbar*vf*(sin(2*x)*D(z) + cos(2*x)*D(y) + D(y))/(2*cos(x))]])

In [101]:
P**-1@Hweyl@P

Matrix([
[hbar*vf*cos(x)*(D(x) + I*D(y))/2 + I*hbar*vf*(2*sin(x) - cos(x)**2 + 2)*D(z)/(2*sin(x) + 2) - I*cos(x)*(hbar*vf*cos(x)*D(z)/2 - I*hbar*vf*(2*sin(x) - cos(x)**2 + 2)*(D(x) - I*D(y))/(2*sin(x) + 2))/(sin(x) + 1), hbar*vf*cos(x)*(D(x) + I*D(y))/2 + I*hbar*vf*(2*sin(x) - cos(x)**2 + 2)*D(z)/(2*sin(x) + 2) - I*cos(x)*(hbar*vf*cos(x)*D(z)/2 - I*hbar*vf*(2*sin(x) - cos(x)**2 + 2)*(D(x) - I*D(y))/(2*sin(x) + 2))/(sin(x) - 1)],
[                                -hbar*vf*cos(x)*(D(x) + I*D(y))/2 + I*hbar*vf*cos(x)**2*D(z)/(2*sin(x) + 2) - I*cos(x)*(-hbar*vf*cos(x)*D(z)/2 - I*hbar*vf*cos(x)**2*(D(x) - I*D(y))/(2*sin(x) + 2))/(sin(x) + 1),                                 -hbar*vf*cos(x)*(D(x) + I*D(y))/2 + I*hbar*vf*cos(x)**2*D(z)/(2*sin(x) + 2) - I*cos(x)*(-hbar*vf*cos(x)*D(z)/2 - I*hbar*vf*cos(x)**2*(D(x) - I*D(y))/(2*sin(x) + 2))/(sin(x) - 1)]])

In [78]:
display(P)
display(D)
reconstructed = P@D@(P)**-1
display(simplify(reconstructed))
HB.equals(reconstructed)

Matrix([
[-I*cos(x)/(sin(x) + 1), -I*cos(x)/(sin(x) - 1)],
[                     1,                      1]])

Matrix([
[-J, 0],
[ 0, J]])

Matrix([
[   J*sin(x), I*J*cos(x)],
[-I*J*cos(x),  -J*sin(x)]])

True

In [110]:
import sympy
from sympy.core.decorators import call_highest_priority
from sympy import Expr, Matrix, Mul, Add, diff
from sympy.core.numbers import Zero

class D(Expr):
    _op_priority = 11.
    is_commutative = False
    def __init__(self, *variables, **assumptions):
        super(D, self).__init__()
        self.evaluate = False
        self.variables = variables

    def __repr__(self):
        return 'D%s' % str(self.variables)

    def __str__(self):
        return self.__repr__()

    @call_highest_priority('__mul__')
    def __rmul__(self, other):
        return Mul(other, self)

    @call_highest_priority('__rmul__')
    def __mul__(self, other):
        if isinstance(other, D):
            variables = self.variables + other.variables
            return D(*variables)
        if isinstance(other, Matrix):
            other_copy = other.copy()
            for i, elem in enumerate(other):
                other_copy[i] = self * elem
            return other_copy

        if self.evaluate:
            return diff(other, *self.variables)
        else:
            return Mul(self, other)

    def __pow__(self, other):
        variables = self.variables
        for i in range(other-1):
            variables += self.variables
        return D(*variables)

def mydiff(expr, *variables):
    if isinstance(expr, D):
        expr.variables += variables
        return D(*expr.variables)
    if isinstance(expr, Matrix):
        expr_copy = expr.copy()
        for i, elem in enumerate(expr):
            expr_copy[i] = diff(elem, *variables)
        return expr_copy
    return diff(expr, *variables)

def evaluateMul(expr):
    end = 0
    if expr.args:
        if isinstance(expr.args[-1], D):
            if len(expr.args[:-1])==1:
                cte = expr.args[0]
                return Zero()
            end = -1
    for i in range(len(expr.args)-1+end, -1, -1):
        arg = expr.args[i]
        if isinstance(arg, Add):
            arg = evaluateAdd(arg)
        if isinstance(arg, Mul):
            arg = evaluateMul(arg)
        if isinstance(arg, D):
            left = Mul(*expr.args[:i])
            right = Mul(*expr.args[i+1:])
            right = mydiff(right, *arg.variables)
            ans = left * right
            return evaluateMul(ans)
    return expr

def evaluateAdd(expr):
    newargs = []
    for arg in expr.args:
        if isinstance(arg, Mul):
            arg = evaluateMul(arg)
        if isinstance(arg, Add):
            arg = evaluateAdd(arg)
        if isinstance(arg, D):
            arg = Zero()
        newargs.append(arg)
    return Add(*newargs)

#courtesy: https://stackoverflow.com/a/48291478/1429450
def disableNonCommutivity(expr):
    replacements = {s: sympy.Dummy(s.name) for s in expr.free_symbols}
    return expr.xreplace(replacements)

def evaluateExpr(expr):
    if isinstance(expr, Matrix):
        for i, elem in enumerate(expr):
            elem = elem.expand()
            expr[i] = evaluateExpr(elem)
        return disableNonCommutivity(expr)
    expr = expr.expand()
    if isinstance(expr, Mul):
        expr = evaluateMul(expr)
    elif isinstance(expr, Add):
        expr = evaluateAdd(expr)
    elif isinstance(expr, D):
        expr = Zero()
    return disableNonCommutivity(expr)